In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# %cd gdrive/MyDrive/HMM/

In [3]:
import numpy as np
from scipy.stats import norm
import scipy as sp

In [4]:
observations = []
N = 0
states = []
transition_matrix = []
emission_matrix = []
means = []
standard_deviations = []



In [5]:
with open('data.txt') as file:
    for readline in file:
        observations.append(float(readline.strip()))

observations = np.array(observations)

# print(observations)

with open('parameters.txt') as file:
    N = int(file.readline())

    for i in range(N):
        states.append(i)

    for i in range(N):
        line = file.readline()
        temp = []
        for w in line.split():
            temp.append(float(w.strip()))
        
        transition_matrix.append(temp)

    line = file.readline()
    for w in line.split():
        means.append(float(w.strip()))

    line = file.readline()
    for w in line.split():
        standard_deviations.append(float(w.strip()))

real_states = ['EL Nino', 'La Nina']
means = np.array(means)
standard_deviations = np.array(standard_deviations)

In [6]:
def initial_prob(transition_matrix):
    lam, vec = sp.linalg.eig(transition_matrix, left=True, right=False)
    # print(lam, vec)
    evec1 = vec[:,np.isclose(lam, 1)]
    # print(evec1)
    evec1 = evec1[:,0]
    # print(evec1)
    pi = evec1/evec1.sum()
    return pi

pi = initial_prob(transition_matrix)


[[-0.31622777]
 [-0.9486833 ]]
[-0.31622777 -0.9486833 ]


In [147]:
def gaussian_distribution(x, mean, std):
    p = norm.pdf(x, loc=mean, scale=np.sqrt(std))
    if p == 0.0:
        p = 1e-323
    return p


for st in states:
    temp = []
    for o in observations:
        p = gaussian_distribution(o, means[st], standard_deviations[st])
        temp.append(p)
    
    emission_matrix.append(temp)


In [148]:
def viterbi(observations, states, pi, transition_matrix, emission_matrix):
    length = len(observations)
    # print(length)
    probability_table = np.array([[float(0)]*length]*N)
    previous_table = np.array([[-1]*length]*N)

    # first entry from stationary probability, without transition
    for st in states:
        # probability_table[st][0] = np.log(pi[st] * gaussian_distribution(observations[0], means[st], standard_deviations[st]))
        probability_table[st][0] = np.log(pi[st] * emission_matrix[st][0])

    
    for i in range(1, length):
        
        for current in states:
            max_incoming_prob = float('-inf')
            prev_selected = -1

            for prev in states:
                incoming_prob = probability_table[prev][i-1] + np.log(transition_matrix[prev][current])
                
                if incoming_prob > max_incoming_prob:
                    max_incoming_prob = incoming_prob
                    prev_selected = prev

            
            # max_prob = max_incoming_prob + np.log(gaussian_distribution(observations[i], means[current], standard_deviations[current]))
            max_prob = max_incoming_prob + np.log(emission_matrix[current][i])

            probability_table[current][i] = max_prob
            previous_table[current][i] = prev_selected

    # print(probability_table)
    # print(previous_table)
    # print(np.argmax(probability_table))

    max = float('-inf')
    best = -1

    most_likely_seuquence = []
    prev = -1

    for i in range(N):
        if probability_table[i][length-1] > max:
            max = probability_table[i][length-1]
            best = i

    most_likely_seuquence.append(best)
    prev = best

    # print(probability_table.shape)
    for i in range(length-2, -1, -1):
        most_likely_seuquence.insert(0, previous_table[prev][i+1])
        prev = previous_table[prev][i+1]

    # print((most_likely_seuquence))

    return most_likely_seuquence


In [149]:
c_t = np.array([0.0]*len(observations))

def forward_algorithm(transition_matrix, emission_matrix, pi):
    length = len(observations)
    alpha = np.array([[0.0]*length]*N)

    for i in range(N):
        alpha[i][0] = pi[i] * emission_matrix[i][0]
        # c_t[0] += alpha[i][0]

    c_t[0] = 1.0/np.sum(alpha[:, 0])
    for i in range(N):
        alpha[i][0] *= c_t[0]
            

    for j in range(1, length):
        
        for i in range(N):
            for k in range(N):
                temp = alpha[k][j-1] * transition_matrix[k][i] * emission_matrix[i][j]
                alpha[i][j] += temp
            
            # c_t[j] += alpha[i][j]

        c_t[j] = 1.0/np.sum(alpha[:, j])
        for i in range(N):
            alpha[i][j] *= c_t[j]

        # dekho thik ache naki
    
    
    return alpha


def backward_algorithm(transition_matrix, emission_matrix):
    length = len(observations)
    beta = np.array([[0.0]*length]*N)

    for i in range(N):
        beta[i][length-1] = 1.0

    for j in range(length-2, -1, -1):
        
        for i in range(N):
            for k in range(N):
                temp = beta[k][j+1] * transition_matrix[i][k] * emission_matrix[k][j+1]
                beta[i][j] += temp
            
            beta[i][j] *= c_t[j]
            # c_t[j] += alpha[i][j]

    
    
    return beta


In [150]:
def maximization_step(alpha, beta):
    length = len(observations)
    global transition_matrix_updated
    global emission_matrix_updated
    global mu_updated
    global sigma_updated

    for i in states:
        for j in states:
            denominator = 0.0
            numerator = 0.0
            
            for k in range(length-1):

                numerator += alpha[i][k] * beta[j][k+1] * transition_matrix_updated[i][j] * emission_matrix_updated[j][k+1]

                denominator += alpha[i][k] * beta[i][k] / c_t[k]

            transition_matrix_updated[i][j] = numerator / denominator

    # print(transition_matrix_updated)

    joint_alpha_beta = alpha * beta

    # print(joint_alpha_beta)

    for col in range(length):
        joint_alpha_beta[:, col] /= joint_alpha_beta[:, col].sum()

    # print(joint_alpha_beta)

    for st in states:
        temp = joint_alpha_beta[st] * observations
        mu_updated[st] = np.sum(temp) / np.sum(joint_alpha_beta[st])

    # print(mu_updated)

    for st in states:
        diff = observations - mu_updated[st]
        temp = np.sum(joint_alpha_beta[st] * diff * diff) / np.sum(joint_alpha_beta[st])
        sigma_updated[st] = np.sqrt(temp)

    # print(sigma_updated)

    
    for st in states:
        for j in range(len(observations)):
            emission_matrix_updated[st][j]  = gaussian_distribution(observations[j], mu_updated[st], sigma_updated[st])


In [151]:
transition_matrix_updated = []
emission_matrix_updated = []
mu_updated = np.array([0.0]*N)
sigma_updated = np.array([0.0]*N)


transition_matrix_updated = np.copy(transition_matrix)
emission_matrix_updated = np.copy(emission_matrix)

In [152]:
def baum_welch():
    
    global transition_matrix_updated
    global emission_matrix_updated
    global sigma_updated
    global mu_updated
    global pi
    global pi_updated

    transition_temp = np.copy(transition_matrix_updated)
    mu_temp = np.copy(mu_updated)
    sigma_temp = np.copy(sigma_updated)

    count = 0
    for _ in range(20):
        pi_updated = initial_prob(transition_matrix_updated)

        alpha = forward_algorithm(transition_matrix_updated, emission_matrix_updated, pi_updated)
        beta = backward_algorithm(transition_matrix_updated, emission_matrix_updated)
        # print(beta)

        maximization_step(alpha, beta)

        flag1 = np.amax(transition_matrix_updated - transition_temp) < 1e-10
        flag2 = np.amax(mu_updated - mu_temp) < 1e-10
        flag3 = np.amax(sigma_updated - sigma_temp) < 1e-10

        count += 1
        
        if flag1 and flag2 and flag3:
            break

        transition_temp = np.copy(transition_matrix_updated)
        mu_temp = np.copy(mu_updated)
        sigma_temp = np.copy(sigma_updated)
        

    print('count' ,count)
    sigma_updated = np.power(sigma_updated, 2)

    return


In [153]:
def viterbi_output(sequence, filename):
    global real_states
    f = open(filename, "w")

    counts = [0]*N

    for s in sequence:
        counts[s] += 1
        f.write('\"' + real_states[s] + '\"\n')

    f.close()

    print(counts)

In [154]:
pi = initial_prob(transition_matrix)

sequence = viterbi(observations, states, pi, transition_matrix, emission_matrix)

filename = 'viterbi_wo_learning.txt'

viterbi_output(sequence, filename)


baum_welch()


print(transition_matrix_updated)
print(mu_updated)
print(sigma_updated)
print(pi)
print(pi_updated)

filename = 'parameters_learned.txt'
f = open(filename, "w")
f.write(str(N) + '\n')
for i in states:
    for j in states:
        f.write(str(round(transition_matrix_updated[i][j], 6)) + "\t")
    
    f.write("\n")

for i in states:
    f.write(str(round(mu_updated[i], 4)) + "\t")

f.write("\n")

for i in states:
    f.write(str(round(sigma_updated[i], 6)) + "\t")

f.write("\n")

for i in states:
    f.write(str(round(pi[i], 6)) + "\t")

f.write("\n")

for i in states:
    f.write(str(round(pi_updated[i], 6)) + "\t")

f.write("\n")

f.close()


filename = "viterbi_after_learning.txt"

sequence = viterbi(observations, states, pi, transition_matrix_updated, emission_matrix_updated)

viterbi_output(sequence, filename)


[291, 709]
[[1.02592878e+00 3.55257946e-01 3.30205665e-01 ... 3.03684775e+51
  6.54926795e-01 1.00000000e+00]
 [3.07778633e+00 1.06577384e+00 9.90616994e-01 ... 9.11054324e+51
  1.96478038e+00 1.00000000e+00]]
[[1.20481750e-02 9.92182150e-01 9.87326155e-02 ... 2.96869180e-03
  1.03317687e+01 1.00000000e+00]
 [1.74369931e-02 1.43595800e+00 1.42893005e-01 ... 4.29650619e-03
  1.49528853e+01 1.00000000e+00]]
[[ 1.27362551  0.26534598  0.16319855 ...  0.66213067  5.16748618
   1.        ]
 [ 5.72439017  1.19261425  0.73350617 ...  2.97598807 23.22559255
   1.        ]]
[[ 6.27746252  0.28128077  0.17865792 ...  0.82239777 11.86242263
   1.        ]
 [28.54483914  1.27903819  0.8123922  ...  3.73960212 53.94073556
   1.        ]]
count 4
[[0.82795699 0.17204301]
 [0.21768707 0.78231293]]
[150.1898689  100.20940296]
[5.031877   8.71346514]
[0.25 0.75]
[0.55855856 0.44144144]
[558, 442]
